### SCS_3252 Big Data Tools - Term Project
Toronto Bike Share is a bicycle sharing and rental system located in the city of Toronto, Ontario, Canada. It was launched in the year 2011 by Public Bike Share Company (PBSC), a privately held company headquartered in Quebec, Canada and acquired by the Toronto Parking Authority (TPA) in 2014 (Marketwired.com, 2014). The service currently operates 270 stations and 2750 bicycles (Bikesharetoronto.com, 2018) and is a popular way of getting around the city. 


This project explores historical bicycle ridership data provided by Bike Share Toronto for temporal and geospatial patterns and trends and examines the possibility, with the addition of external data such as the temperature, precipitation and other weather conditions, of building models to predict the total number of rides that is likely to occur on a given day.


<b>Data Sources:</b>

 - Ridership Data: Historical ridership data was sourced from the City of Toronto’s Open Data Portal website from the following three Excel workbooks:
  - <a href="https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_Bike_Share_Toronto_Ridership_Q3.xlsx">2016 Q3 Data</a>: contains ridership data for the months of July, August and September 2016 
  - <a href="https://www.toronto.ca/ext/open_data/catalog/data_set_files/2016_Bike_Share_Toronto_Ridership_Q4.xlsx">2016 Q4 Data</a>: contains ridership data for the months of October, November and December 2016
  - <a href="https://www.toronto.ca/ext/open_data/catalog/data_set_files/2014Q4_to_2015Q3_Bike_Share_Toronto_Ridership.xlsx">2014 Q4 – 2015 Q3 Data</a>: contains ridership data (only at a summary level) for the time period between 2014 October and 2015 September


 - Information on station locations was obtained as a JSON feed from this <a href ="https://tor.publicbikesystem.net/ube/gbfs/v1/en/station_information">URL</a>. 
 
 
 - Historical weather information for the City of Toronto for the years 2015 and 2016 was obtained from the from the <a href ="http://climate.weather.gc.ca/historical_data/search_historic_data_e.html?Month=7&Day=29&Year=2018&timeframe=2&StartYear=1840&EndYear=2018">Government of Canada website</a>. This included information such as the daily minimum maximum and mean temperatures, wind speeds, precipitation levels etc.


### Import Libraries

In [1]:
import pandas as pd
import json
from pprint import pprint
from pandas.io.json import json_normalize
import time
import re
import numpy as np
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import calendar
import plotly.figure_factory as ff

import datetime

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

### Data Paths

In [2]:
#pathToDataFiles = '../Data/'
#pathToInputDataFiles = 'Data/'
#pathToOutputDataFiles = ''

pathToInputDataFiles = '../Data/'
pathToOutputDataFiles = '../'

### Generate a JSON file of stations data

In [3]:
with open(pathToInputDataFiles + 'station_information.json') as f:
    stationData = json.load(f)

# Get station info in a dataframe for later use
stationsDF = json_normalize(stationData['data']['stations'])

stationGeoJson = json.dumps({ 'type': 'FeatureCollection',
                        'features': [ 
                                        {'type': 'Feature',
                                         'geometry': { 'type': 'Point',
                                                       'coordinates': [ station['lon'],
                                                                        station['lat']]},
                                         'properties': { key: value 
                                                         for key, value in station.items()
                                                         if key not in ('lat', 'lon') }
                                         } 
                                     for station in stationData['data']['stations']
                                    ]
                       })
with open(pathToOutputDataFiles+'stationLocations.geojson', 'w') as outfile:
    outfile.write('var stationLocations = ' + stationGeoJson + ';')

### Read in all trips data files for 2015/2016

In [4]:
#trips_Q4_2016 = pd.read_excel('../Data/2016_Bike_Share_Toronto_Ridership_Q4.xlsx')
trips_Q4_2016 = pd.read_excel(pathToInputDataFiles + '2016_Bike_Share_Toronto_Ridership_Q4.xlsx')
trips_Q3_2016 = pd.read_excel(pathToInputDataFiles + '2016_Bike_Share_Toronto_Ridership_Q3.xlsx')
trips_Q1_Q2_2016=pd.read_csv(pathToInputDataFiles+'2016_Q1_Q2-Total_Rides.csv')

### Clean-up datasets before merging them into a single one

In [5]:
# The Q4-2016 dataset seems to have some rows where dates are encoded as yyyy-dd-mm.
# We will change these to yyyy-mm-dd for consistency
timeStampFixer = (lambda ts: pd.Timestamp(year=ts.year, 
                        #Flip month and day to fix this problem
                        month=ts.day, 
                        day=ts.month, 
                        hour=ts.hour, 
                        minute=ts.minute, 
                        second=ts.second) if ts.month < 10 else ts)
trips_Q4_2016['trip_start_time'] = list(map(timeStampFixer, trips_Q4_2016['trip_start_time']))
trips_Q4_2016['trip_stop_time'] = list(map(timeStampFixer, trips_Q4_2016['trip_stop_time']))

# Drop the one row that has a date of 2000-01-01 which does not fall within Q3, 2016
trips_Q3_2016 = trips_Q3_2016[~(trips_Q3_2016['trip_start_time'] < '2016-01-01 00:00:00')]

### Merge Datasets into a single one and cast timestamps to UTC before converting them to US/Eastern

In [6]:
allTrips2015_2016 = trips_Q3_2016.append(trips_Q4_2016)
allTrips2015_2016['trip_start_time'] = allTrips2015_2016['trip_start_time'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')
allTrips2015_2016['trip_stop_time'] = allTrips2015_2016['trip_stop_time'].dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

### Add new columns

In [7]:
# Add a Trip Date column from the start timestamp
allTrips2015_2016['trip_date']=list(map (lambda x: x[0:10], allTrips2015_2016['trip_start_time'].astype(str)))

# Add the unix timestamp representation of trip start times
allTrips2015_2016['trip_start_time_unix'] = allTrips2015_2016['trip_start_time'].astype(np.int64)/(10 ** 9)


### Correct mismatched station names

In [8]:
missingStationNames_from = pd.DataFrame(allTrips2015_2016[~allTrips2015_2016['from_station_name'].isin(
    stationsDF['name'])]['from_station_name'].value_counts()).reset_index().drop('from_station_name', 
                                                                                 axis=1).rename(columns={'index':'station_name'})

missingStationNames_to = pd.DataFrame(allTrips2015_2016[~allTrips2015_2016['to_station_name'].isin(
    stationsDF['name'])]['to_station_name'].value_counts()).reset_index().drop('to_station_name', 
                                                                                 axis=1).rename(columns={'index':'station_name'})

missingStationNames = missingStationNames_from.append(missingStationNames_to).drop_duplicates()


missingStationNames['individual_strt_names']=list(map(lambda x: (re.sub(pattern='(\s(W|E|N|S)(\W|$))', 
                                                                        repl='', 
                                                                        string=x, 
                                                                        flags=re.IGNORECASE)).split('/'),
                                                    missingStationNames['station_name']))
missingStationNames['correct_station_name']=''

for index, row in missingStationNames.iterrows():
    indivStrtNames = row['individual_strt_names']
    #print(indivStrtNames)
    for index_1, row_1 in stationsDF.iterrows():
        #print(' -->' + row_1['name'])
        matchFound = True
        for strtName in indivStrtNames:
            if((re.sub(string=strtName.strip(), pattern='(\.|\')', repl='') in re.sub(string=row_1['name'], pattern='(\.|\')', repl='')) == False):
                matchFound = False
        if(matchFound):
            missingStationNames.iloc[index]['correct_station_name'] = row_1['name']

# Make some manual updates          
missingStationNames['correct_station_name'] = np.where(missingStationNames['station_name']=='Wellesley St E / Yonge St Green P', 'Wellesley St E / Yonge St (Green P)',
         np.where(missingStationNames['station_name']=='Landsdowne Subway Green P', 'Lansdowne Subway Green P',
                 np.where(missingStationNames['station_name']== 'Queen St E / Berkely St', 'Queen St E / Berkeley St',
                          np.where(missingStationNames['station_name']=='Bloor GO / UP Station/ Rail Path', 'Bloor GO / UP Station (West Toronto Railpath)',
                                   np.where(missingStationNames['station_name']=='Base Station', 'Base Station',
                                   missingStationNames['correct_station_name'])))))
missingStationNames = missingStationNames[['station_name', 'correct_station_name']]
missingStationsDict = missingStationNames.set_index('station_name').to_dict()['correct_station_name']

stationNameFixer = (lambda stationName: stationName if (missingStationsDict.get(stationName, 'NOT_PRESENT') == 'NOT_PRESENT') 
                    else missingStationsDict.get(stationName))
allTrips2015_2016['from_station_name'] = list(map(stationNameFixer, allTrips2015_2016['from_station_name']))
allTrips2015_2016['to_station_name'] = list(map(stationNameFixer, allTrips2015_2016['to_station_name']))

### Join to the stations dataset

In [9]:
allTripsWithStationInfo2015_2016 = pd.merge(allTrips2015_2016, 
                                  stationsDF[['name', 'lat', 'lon', 'station_id']], 
                                  left_on='from_station_name', 
                                  right_on='name').rename(columns={'lat':'start_lat', 
                                                                   'lon':'start_long',
                                                                   'station_id': 'from_station_id'}).drop('name', axis=1)

allTripsWithStationInfo2015_2016 = pd.merge(allTripsWithStationInfo2015_2016, 
                                  stationsDF[['name', 'lat', 'lon', 'station_id']], 
                                  left_on='to_station_name', 
                                  right_on='name').rename(columns={'lat':'end_lat', 
                                                                   'lon':'end_long',
                                                                   'station_id': 'to_station_id'}).drop('name', axis=1)


In [10]:
specDayTrips = allTripsWithStationInfo2015_2016[allTripsWithStationInfo2015_2016['trip_date']=='2016-09-16'].reset_index(drop=True)


# Drop trips that start and end at the same station
specDayTrips = (specDayTrips[~(specDayTrips['from_station_name'] == specDayTrips['to_station_name'])]).reset_index(drop=True)

# Sort by trip start times
specDayTrips = specDayTrips.sort_values(by='trip_start_time_unix')

# Convert to geojson
trips = []
for index, row in specDayTrips.iterrows():
    trips.append({
            'geometry':{
                'type': 'LineString',
                "coordinates" : [[row['start_long'], row['start_lat']], [row['end_long'], row['end_lat']]]
            },
            'type' : "Feature",
            'properties' : {
                'trip_id': row['trip_id'],
                #'trip_start_time' : str(row['trip_start_time']),
                #'trip_stop_time' : str(row['trip_stop_time']),
                'from_station_id': row['from_station_id'],
                'to_station_id': row['to_station_id'],
                'trip_start_time_unix' : row['trip_start_time_unix'],
                #'trip_stop_time_unix' : row['trip_end_time_unix'],
                'trip_duration_seconds' : row['trip_duration_seconds'],
                'user_type': row['user_type']
            }
        })

tripsGJ = {
        "type": "FeatureCollection",
        "features": trips
    }
tripsGeoJson = json.dumps(tripsGJ)

# Only write first 10 rows
#specDayTrips = specDayTrips[0:10]
with open(pathToOutputDataFiles+'trips.geojson', 'w') as outfile:
    outfile.write('var dayStartTime = '+str(specDayTrips['trip_start_time_unix'].min()) + ';\n')
    outfile.write('var dayEndTime = '+str(specDayTrips['trip_start_time_unix'].max()) + ';\n')
    outfile.write('var bicycleTrips = ' + tripsGeoJson + ';\n')
    

### Read in 2014/2015 data

In [11]:
xls = pd.ExcelFile(pathToInputDataFiles+'2014Q4_to_2015Q3_Bike_Share_Toronto_Ridership.xlsx')

In [12]:
tripSheetLists = ['2014-10 Trips',
                    '2014-11 Trips',
                    '2014-12 Trips',
                    '2015-01 Trips',
                    '2015-02 Routes',
                    '2015-03 Routes',
                    '2015-04 Routes',
                    '2015-05 Routes',
                    '2015-06 Routes',
                    '2015-07 Trips',
                    '2015-08 Trips',
                    '2015-09 Trips']
trips2014_2015 = pd.DataFrame()
for sheetName in tripSheetLists:
    df = pd.read_excel(xls, sheetName, skiprows=[0], skipfooter=1)
    yearMonth = re.sub(string=sheetName, pattern='(\s(Trips|Routes))', repl='')
    
    year = int(yearMonth.split('-')[0])
    month = int(yearMonth.split('-')[1])
    
    df['year_month'] = yearMonth
    df['year'] = year
    df['month'] = month
    
    if (trips2014_2015 is None):
        trips2014_2015 = df
    else:
        trips2014_2015 = trips2014_2015.append(df)
    trips2014_2015.reset_index(drop=True)

# Replace nan with zeroes
trips2014_2015.replace(to_replace=np.NaN, value=0, inplace=True)

# Read in 2014/2015 stations information
stations2014_2015 = pd.read_excel(xls, 'Station Key')
# Update station 7081 to id 7124. Looks like the latest version has a different id for this station
# based on the fact that they have the same address
stations2014_2015['Terminal'] = np.where(stations2014_2015['Terminal']==7081, 
         7124,
         stations2014_2015['Terminal'])

# Read in weekday/weekend trips by hour information
weekdayTripsByHour2014_2015 = pd.read_excel(xls, 'Weekday Trips by Hour', skiprows=[0], skipfooter=1) 
weekendTripsByHour2014_2015 = pd.read_excel(xls, 'Weekend Trips by Hour', skiprows=[0], skipfooter=1) 

### Temporal Trends - Month of year

In [33]:
#Calculate totals for 2014/2015
monthlyCounts2014_2015 = trips2014_2015.groupby('month', as_index=False).agg({'Casual':'sum', 'Registered':'sum', 'Total':'sum'}).rename(columns={'Registered':'Member'})

#Calculate totals for 2015/2016
allTripsWithStationInfo2015_2016['month'] = allTripsWithStationInfo2015_2016['trip_start_time'].dt.month
monthlyCounts2015_2016=allTripsWithStationInfo2015_2016.groupby(
                                            ['month', 
                                             'user_type']).size().reset_index(name='counts').pivot(index='month', 
                                                                                                columns='user_type', 
                                                                                                values='counts').rename_axis(None, 
                                                                                                                             axis=1).reset_index()
monthlyCounts2015_2016 = monthlyCounts2015_2016[~(monthlyCounts2015_2016['month']==6)].reset_index(drop=True)
monthlyCounts2015_2016['Total'] = monthlyCounts2015_2016['Member'] + monthlyCounts2015_2016['Casual']

#Combine all three to obtain the overall monthly summary
allMonthlySummary = monthlyCounts2014_2015[['month', 
                                            'Total']].append(monthlyCounts2015_2016[['month', 
                                                                                     'Total']]).append(trips_Q1_Q2_2016[['month', 
                                                                                     'Total']]).reset_index(drop=True)
# Add month name
allMonthlySummary['month_name']=list(map(lambda x: calendar.month_name[x], allMonthlySummary['month']))
# Compute Average
allMonthlySummary_avgRides = allMonthlySummary.groupby(['month','month_name'], as_index=False).agg({'Total':'mean'})

In [34]:
allMonthlySummary

,month,Total,month_name
0,1,18030,January
1,2,8339,February
2,3,25840,March
3,4,34164,April
4,5,73551,May
5,6,77602,June
6,7,98867,July
7,8,90274,August
8,9,81858,September
9,10,57096,October


In [49]:
# Plot in a chart
trace1 = go.Bar(
    y=allMonthlySummary_avgRides['Total'],
    x='<b>' + allMonthlySummary_avgRides['month_name'] + '</b>',
    #name='Casual Users',
    marker=dict(
        color='#454688'
    )
)
data = [trace1]

layout = go.Layout(
    title='<b>Average Total Trips by Month of Year (Q4-2014 to Q3-2015 & All Quarters 2016)</b>',
    yaxis=dict(
        title='<b>No. of Trips</b>',
        zeroline=False
    )
    #boxmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Temporal Trends - Hour of Day

In [15]:
allTripsWithStationInfo2015_2016['day_of_week'] = allTripsWithStationInfo2015_2016['trip_start_time'].dt.weekday
allTripsWithStationInfo2015_2016['trip_start_hour'] = allTripsWithStationInfo2015_2016['trip_start_time'].dt.hour

weekDaysMap={0: 'Monday',
             1: 'Tuesday',
             2: 'Wednesday',
             3: 'Thursday',
             4: 'Friday',
             5: 'Saturday',
             6: 'Sunday'}

dailySummary2015_2016 = allTripsWithStationInfo2015_2016.groupby(['day_of_week', 'trip_start_hour']).size().reset_index()
dailySummary2015_2016.columns = ['day_of_week', 'trip_start_hour', 'count']
dailySummary2015_2016['day_of_week_str'] = list(map(lambda x: weekDaysMap[x], dailySummary2015_2016['day_of_week']))
dailySummary2015_2016['trip_start_hour_str'] = list(map(lambda x: '' + ('0' if x < 10 else '') + str(x) + ':00', dailySummary2015_2016['trip_start_hour']))


In [50]:
# Plot in a chart
trace1 = go.Heatmap(
    z = dailySummary2015_2016['count'],
    y = '<b>' + dailySummary2015_2016['day_of_week_str'] + '</b>',
    x = '<b>' + dailySummary2015_2016['trip_start_hour_str'] + '</b>',
    xgap = 5,
    ygap = 5,
    colorscale='Viridis'
)
data = [trace1]

layout = go.Layout(
    title='<b>Trip Volumes by Day of Week and Hour of Day (Q3/Q4-2016)</b>',
    yaxis=dict(
        zeroline=False
    ),
    xaxis=dict(
        title='<b>Trip Start Time</b>'
    )
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [55]:
tripDurationDF = allTripsWithStationInfo2015_2016[['month', 
                                                   'trip_duration_seconds', 
                                                   'user_type']].sort_values(by='month')
tripDurationDF['month_name']=list(map(lambda x: calendar.month_name[x], tripDurationDF['month']))

tripDurationDF = tripDurationDF.groupby(['month', 
                                         'month_name', 
                                         'user_type'], as_index=False).agg({'trip_duration_seconds':'mean'}).sort_values(by='month')

tripDurationDF['trip_duration_minutes']=tripDurationDF['trip_duration_seconds']/60
tripDurationDF['trip_duration_minutes']=tripDurationDF['trip_duration_minutes'].astype(np.int64)

# Plot in a chart
trace1 = go.Bar(
    y=tripDurationDF[tripDurationDF['user_type']=='Casual']['trip_duration_minutes'],
    x=tripDurationDF[tripDurationDF['user_type']=='Casual']['month_name'],
    name='Casual Users',
    marker=dict(
        color='#6B77A2'
    )
)
trace2 = go.Bar(
    y=tripDurationDF[tripDurationDF['user_type']=='Member']['trip_duration_minutes'],
    x=tripDurationDF[tripDurationDF['user_type']=='Member']['month_name'],
    name='Members',
    marker=dict(
        color='#FF4136'
    )
)
data = [trace1, trace2]

layout = go.Layout(
    title='<b>Average Trip Duration by Month of Year (Q3/Q4-2016)</b>',
    yaxis=dict(
        title='<b>minutes</b>',
        zeroline=False
    ),
    barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Geospatial Trends

In [18]:
fromStationCounts_raw = allTripsWithStationInfo2015_2016.groupby(['from_station_id']).size().reset_index().rename(columns={0:'Total'}).append(
      trips2014_2015.groupby(['Start Terminal'], as_index=False).agg({'Total':
                                                                      'sum'}).rename(columns={'Start Terminal':
                                                                                              'from_station_id'})).reset_index(drop=True)

fromStationCounts_summary = fromStationCounts_raw.groupby(['from_station_id'], 
                              as_index=False).agg({'Total':'sum'}).sort_values(by='Total', 
                                                                               ascending=False).reset_index(drop=True)

#Top 5 From stations with stations info
fromStationCounts_summary = pd.merge(fromStationCounts_summary, stationsDF, left_on='from_station_id', right_on='station_id')
fromStationCounts_summary[0:5]

,from_station_id,Total,address,capacity,cross_street,lat,lon,name,post_code,rental_methods,station_id
0,7006,24429,Bay St / College St (East Side),11,NaN,43.660439,-79.385525,Bay St / College St (East Side),NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7006
1,7010,21361,King St W / Spadina Ave,19,NaN,43.645323,-79.395003,King St W / Spadina Ave,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7010
2,7038,21205,Dundas St W / Yonge St,15,NaN,43.656094,-79.381484,Dundas St W / Yonge St,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7038
3,7057,21048,Simcoe St / Wellington St W,31,NaN,43.645533,-79.385400,Simcoe St / Wellington St W,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7057
4,7030,19992,Bay St / Wellesley St W,35,null,43.664088,-79.387095,Bay St / Wellesley St W,null,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7030


In [19]:
toStationCounts_raw = allTripsWithStationInfo2015_2016.groupby(['to_station_id']).size().reset_index().rename(columns={0:'Total'}).append(
      trips2014_2015.groupby(['End Terminal'], as_index=False).agg({'Total':
                                                                      'sum'}).rename(columns={'Start Terminal':
                                                                                              'to_station_id'})).reset_index(drop=True)

toStationCounts_summary = toStationCounts_raw.groupby(['to_station_id'], 
                              as_index=False).agg({'Total':'sum'}).sort_values(by='Total', 
                                                                               ascending=False).reset_index(drop=True)
# Top 'To' Stations with stations info
toStationCounts_summary = pd.merge(toStationCounts_summary, stationsDF, left_on='to_station_id', right_on='station_id')
toStationCounts_summary[0:5]

,to_station_id,Total,address,capacity,cross_street,lat,lon,name,post_code,rental_methods,station_id
0,7033,12378,Union Station,27,NaN,43.645609,-79.380386,Union Station,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7033
1,7038,10013,Dundas St W / Yonge St,15,NaN,43.656094,-79.381484,Dundas St W / Yonge St,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7038
2,7010,9754,King St W / Spadina Ave,19,NaN,43.645323,-79.395003,King St W / Spadina Ave,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7010
3,7057,9731,Simcoe St / Wellington St W,31,NaN,43.645533,-79.385400,Simcoe St / Wellington St W,NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7057
4,7016,9505,Bay St / Queens Quay W (Ferry Terminal),23,NaN,43.640823,-79.376265,Bay St / Queens Quay W (Ferry Terminal),NaN,"[KEY, CREDITCARD, TRANSITCARD, PHONE]",7016


In [20]:
# Compute total trip counts by route for all data
routeTripSummary_raw = trips2014_2015.groupby(['Start Terminal', 
                        'End Terminal'], as_index=False).agg({'Total':'sum'}).rename(columns={'Start Terminal' :'from_station_id', 
                        'End Terminal':'to_station_id'}).append(
            allTripsWithStationInfo2015_2016.groupby(['from_station_id', 
                                                      'to_station_id']).size().reset_index().rename(columns={0:'Total'}))
routeTripSummary = routeTripSummary_raw.groupby(['from_station_id',
                              'to_station_id'], as_index=False).agg({'Total':'sum'}).sort_values(by='Total', 
                                                                                                 ascending=False).reset_index(drop=True)


In [21]:
#routeTripSummary.pivot(index='from_station_id', columns='to_station_id', values='Total')
routeTripSummary = pd.merge(routeTripSummary, 
         stationsDF[['station_id', 'name']], 
         left_on='from_station_id', 
         right_on='station_id').rename(columns={'name':'from_station_name'}).drop('station_id', axis=1)

routeTripSummary = pd.merge(routeTripSummary, 
         stationsDF[['station_id', 'name']], 
         left_on='to_station_id', 
         right_on='station_id').rename(columns={'name':'to_station_name'}).drop('station_id', axis=1)
routeTripSummary = routeTripSummary.sort_values(by='Total', ascending=False).reset_index(drop=True)

routeTripSummary[0:5]

,from_station_id,to_station_id,Total,from_station_name,to_station_name
0,7016,7016,4006,Bay St / Queens Quay W (Ferry Terminal),Bay St / Queens Quay W (Ferry Terminal)
1,7028,7048,2766,Gould St / Mutual St,Front St W / Yonge St (Hockey Hall of Fame)
2,7048,7028,2230,Front St W / Yonge St (Hockey Hall of Fame),Gould St / Mutual St
3,7076,7076,1858,York St / Queens Quay W,York St / Queens Quay W
4,7057,7010,1709,Simcoe St / Wellington St W,King St W / Spadina Ave


In [22]:
# Find top 10 from and to stations
top10FromAndToStations = set(np.concatenate((fromStationCounts_summary['from_station_id'][0:10].values,
toStationCounts_summary['to_station_id'][0:10].values.astype(np.int64)), axis=0))

routeTripSummary['from_station_name'] = np.where(routeTripSummary['from_station_id'].isin(top10FromAndToStations),
         routeTripSummary['from_station_name'],
         'Other')
routeTripSummary['to_station_name'] = np.where(routeTripSummary['to_station_id'].isin(top10FromAndToStations),
         routeTripSummary['to_station_name'],
         'Other')

routeTripSummary = routeTripSummary.groupby(['from_station_name', 'to_station_name'], as_index=False).agg({'Total':'sum'})
#routeTripSummary = routeTripSummary[~(routeTripSummary['from_station_name'] == routeTripSummary['to_station_name'])]

routeTripSummary_pivot = routeTripSummary.pivot(index='from_station_name', columns='to_station_name', values='Total').rename_axis(None, axis=1).rename_axis(None, axis=0)
routeTripSummary_pivot.replace(to_replace=np.NaN, value=0, inplace=True)



### Build a prediction model to estimate monthly ridership

In [23]:
trips2014_2015[['year', 'month', 'Total']].groupby(['year', 'month'], as_index=False).agg({'Total':'sum'})
trips_Q1_Q2_2016

,year,month,Total
0,2016,1,22000
1,2016,2,21000
2,2016,3,30000
3,2016,4,41000
4,2016,5,70000
5,2016,6,80000


In [24]:
def cleanUpWeatherData(weatherData):
    df=weatherData.copy()
    df = df[['Year', 'Month', 'Day', 'Max Temp (C)', 'Min Temp (C)', 'Mean Temp (C)', 'Total Precip (mm)', 'Spd of Max Gust (km/h)']]
    df['Spd of Max Gust (km/h)'] = list(map(lambda x: 0 if pd.isnull(x) else int(re.sub(pattern='[^0-9.]', string=x, repl='')), df['Spd of Max Gust (km/h)']))
    return df

weather2015 = cleanUpWeatherData(pd.read_csv(pathToInputDataFiles+'2015-weather.csv'))
weather2016 = cleanUpWeatherData(pd.read_csv(pathToInputDataFiles+'2016-weather.csv'))

allWeatherData = (weather2015.append(weather2016)).reset_index(drop=True)

mthlyAverages = allWeatherData.dropna().reset_index(drop=True).groupby(['Year', 'Month'], as_index=False).agg('mean').drop('Day', axis=1)
mthlyAverages.columns = ['Year', 'Month'] + [col + '_mean' for col in mthlyAverages if col not in ['Year', 'Month']]

allWeatherData = pd.merge(allWeatherData, mthlyAverages, on=['Year', 'Month'])

colList = ['Max Temp (C)',
           'Min Temp (C)',
           'Mean Temp (C)', 
           'Total Precip (mm)', 
           'Spd of Max Gust (km/h)']

weatherValFixer = (lambda x,y: (y if pd.isnull(x) else x))

for col in colList:
    allWeatherData[col]=list(map(weatherValFixer, allWeatherData[col], allWeatherData[col+'_mean']))

allWeatherData = allWeatherData.drop([col + '_mean' for col in colList], axis=1)

In [25]:
allTripsWithStationInfo2015_2016['year'] = allTripsWithStationInfo2015_2016['trip_start_time'].dt.year
allTripsWithStationInfo2015_2016['day'] = allTripsWithStationInfo2015_2016['trip_start_time'].dt.day
tripsDataForPred = allTripsWithStationInfo2015_2016[['year', 'month', 'day', 'day_of_week']]
tripsDataForPred = tripsDataForPred.groupby(['year', 'month', 'day', 'day_of_week']).size().reset_index().rename(columns={0:'Total'})

# Remve June 2016 since we don't have the full set of data for this month
tripsDataForPred = tripsDataForPred[~((tripsDataForPred['year'] == 2016) & (tripsDataForPred['month'] == 6))].reset_index(drop=True)

# Join to weather data
tripsDataForPred = pd.merge(tripsDataForPred, allWeatherData, left_on=['year',
                                                    'month',
                                                    'day'], right_on=['Year', 
                                                                      'Month', 
                                                                      'Day']).drop(['Year', 'Month', 'Day'], axis=1)

# One-hot encode month and day of week
months_oneHot = pd.get_dummies(tripsDataForPred['month'])
months_oneHot.columns = ['month_' + str(col) for col in months_oneHot.columns]

tripsDataForPred = tripsDataForPred.join(months_oneHot)

dayOfWeek_ontHot = pd.get_dummies(tripsDataForPred['day_of_week'])
dayOfWeek_ontHot.columns = ['DoW_' + str(col) for col in dayOfWeek_ontHot.columns]
tripsDataForPred = tripsDataForPred.join(dayOfWeek_ontHot)


### Linear Regression Model

In [26]:
# Split into training and test sets
train, test = train_test_split(tripsDataForPred, test_size=0.2)

# Extract features
train_X = train[[col for col in train.columns.values if ((col not in ['Total','year','month','day_of_week']))]]
test_X = test[[col for col in test.columns.values if ((col not in ['Total','year','month','day_of_week']))]]

# Extract dependent variable
train_Y = train['Total']
test_Y = test['Total']

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(train_X, train_Y)

# Make predictions using the testing set
linReg_Test_Predictions = regr.predict(test_X)
linReg_Train_Predictions = regr.predict(train_X)

# The coefficients
#print('Coefficients: \n', regr.coef_)
coeffs = zip(train_X.columns, regr.coef_)
coeffs = sorted(coeffs, key = lambda x: np.absolute(x[1]), reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Coefficient: {}'.format(*pair)) for pair in coeffs];

# Calculate the absolute errors
linReg_errors = abs(linReg_Train_Predictions - train_Y)

# Print out the mean absolute error (mae)
print('\nMean Absolute Error:', round(np.mean(linReg_errors), 2), ' trips\n')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(train_Y, linReg_Train_Predictions))


Variable: month_12             Coefficient: -1227.273074905115
Variable: month_9              Coefficient: 1063.484072707216
Variable: Mean Temp (C)        Coefficient: -908.9868391467323
Variable: DoW_6                Coefficient: -641.4124513313898
Variable: month_8              Coefficient: 603.565738512465
Variable: Min Temp (C)         Coefficient: 487.6805010179473
Variable: Max Temp (C)         Coefficient: 463.69565624669104
Variable: DoW_5                Coefficient: -433.52867704896045
Variable: month_11             Coefficient: -364.2472724836395
Variable: DoW_1                Coefficient: 337.58457506304313
Variable: DoW_3                Coefficient: 291.54612162694457
Variable: DoW_2                Coefficient: 233.60682283950865
Variable: DoW_4                Coefficient: 172.1089214446979
Variable: month_10             Coefficient: -54.43079556883197
Variable: Total Precip (mm)    Coefficient: -52.7562580043508
Variable: DoW_0                Coefficient: 40.0946874061482

In [56]:
linearReg_Test_results = test.copy()
linearReg_Test_results['Predicted'] = linReg_Test_Predictions

# Plot in a chart
linearReg_Test_results['trip_date'] = list(map(lambda x, y, z: datetime.datetime(year=x, month=y, day=z),
                linearReg_Test_results['year'], 
                linearReg_Test_results['month'], 
                linearReg_Test_results['day']))
linearReg_Test_results = linearReg_Test_results.sort_values('trip_date')

# Plot in a chart
trace1 = go.Scatter(
    y=linearReg_Test_results['Total'],
    x=linearReg_Test_results['trip_date'],
    name='Actual',
    marker=dict(
        color='#6B77A2'
    )
)
trace2 = go.Scatter(
    y=linearReg_Test_results['Predicted'],
    x=linearReg_Test_results['trip_date'],
    name='Predicted',
    marker=dict(
        color='#FF4136'
    )
)
data = [trace1, trace2]

layout = go.Layout(
    title='<b>Linear Regression Model [Predicted vs. Actual] (Q3/Q4-2016)</b>',
    yaxis=dict(
        title='<b>No. of Trips</b>'
    )
    #barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)

### Fit a RandomForest Regressor

In [28]:
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(train_X, train_Y)

# Use the forest's predict method on the test data
rf_Train_Predictions = rf.predict(train_X)
rf_Test_Predictions = rf.predict(test_X)

# Calculate the absolute errors
rf_errors = abs(rf_Train_Predictions - train_Y)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(rf_errors), 2), ' trips')

# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(train_Y, rf_Train_Predictions))

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(train_X.columns, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Mean Absolute Error: 190.17  trips
Variance score: 0.96
Variable: Max Temp (C)         Importance: 0.44
Variable: Mean Temp (C)        Importance: 0.16
Variable: day                  Importance: 0.09
Variable: Min Temp (C)         Importance: 0.07
Variable: Total Precip (mm)    Importance: 0.04
Variable: Spd of Max Gust (km/h) Importance: 0.04
Variable: month_9              Importance: 0.03
Variable: DoW_6                Importance: 0.03
Variable: month_7              Importance: 0.02
Variable: month_12             Importance: 0.02
Variable: DoW_5                Importance: 0.02
Variable: month_8              Importance: 0.01
Variable: month_10             Importance: 0.01
Variable: DoW_1                Importance: 0.01
Variable: DoW_2                Importance: 0.01
Variable: month_11             Importance: 0.0
Variable: DoW_0                Importance: 0.0
Variable: DoW_3                Importance: 0.0
Variable: DoW_4                Importance: 0.0


In [58]:
rf_Test_Results = test.copy()
rf_Test_Results['Predicted']= (rf_Test_Predictions.astype(np.int64))

# Plot in a chart
rf_Test_Results['trip_date'] = list(map(lambda x, y, z: datetime.datetime(year=x, month=y, day=z),
                rf_Test_Results['year'], 
                rf_Test_Results['month'], 
                rf_Test_Results['day']))
rf_Test_Results = rf_Test_Results.sort_values('trip_date')

# Plot in a chart
trace1 = go.Scatter(
    y=rf_Test_Results['Total'],
    x=rf_Test_Results['trip_date'],
    name='Actual',
    marker=dict(
        color='#6B77A2'
    )
)
trace2 = go.Scatter(
    y=rf_Test_Results['Predicted'],
    x=rf_Test_Results['trip_date'],
    name='Predicted',
    marker=dict(
        color='#FF4136'
    )
)
data = [trace1, trace2]

layout = go.Layout(
    title='<b>RandomForest Regression Model [Predicted vs. Actual] (Q3/Q4-2016)</b>',
    yaxis=dict(
        title='<b>No. of Trips</b>'
    )
    #barmode='group'
)
fig = go.Figure(data=data, layout=layout)
iplot(fig)